Data cleaning & preprocessing

Import libraries

In [284]:
import numpy as np
import pandas as pd

from sklearn.impute import KNNImputer

import matplotlib.pyplot as plt

#supress warnings related to libraries
import warnings
warnings.filterwarnings("ignore")

Read the Data from CSV file in Visual Studio Code (VSC)

In [285]:
df =pd.read_csv("/workspaces/SDS-009-ds-salary-pred/data/salaries.csv", skipinitialspace=True)
#General structure of DS
df.info()
#check if there are any null or missing values
print(df.isna().sum())
# %% of DS
#print(df.isna().sum()/len(df)*100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        868 non-null    object 
 1   Company Score  789 non-null    float64
 2   Job Title      870 non-null    object 
 3   Location       857 non-null    object 
 4   Date           870 non-null    object 
 5   Salary         764 non-null    object 
dtypes: float64(1), object(5)
memory usage: 40.9+ KB
Company            2
Company Score     81
Job Title          0
Location          13
Date               0
Salary           106
dtype: int64


Strip before and after Whitespaces from entire data frame records

In [286]:
#Remove leading and following spaces in data
# #strip to all rows
df =df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

#strip to all columns
df.columns = df.columns.str.strip()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        868 non-null    object 
 1   Company Score  789 non-null    float64
 2   Job Title      870 non-null    object 
 3   Location       857 non-null    object 
 4   Date           870 non-null    object 
 5   Salary         764 non-null    object 
dtypes: float64(1), object(5)
memory usage: 40.9+ KB


Unfold Salary column

In [287]:
df['MIN'] = df['Salary'].str.extract('(\d{1,4})')
df['MAX'] = df['Salary'].str.extract('(.-.\$\d{1,4}K)')
df['MAX'] = df['MAX'].str.extract('(\d{1,4})')
df['Hourly'] = df['Salary'].str.extract('(Per Hour)')
df['Source'] = df['Salary'].str.extract('(\([a-zA-Z]+)')
df['Source'] = df['Source'].str.extract('([a-zA-Z]+)')
df.sample(100)

,Company,Company Score,Job Title,Location,Date,Salary,MIN,MAX,Hourly,Source
14,Athenahealth,4.0,"Software Engineer, athenaCollector – RCM Rules","Boston, MA",2d,$192K - $288K (Employer est.),192,288,NaN,Employer
818,Coupang,NaN,Analytics Engineer,"Los Angeles, CA",3d,NaN,NaN,NaN,NaN,NaN
175,NBCUniversal,4.4,Post Software Engineer,"Englewood Cliffs, NJ",3d,$145K - $162K (Employer est.),145,162,NaN,Employer
537,"Grammarly, Inc.",4.0,Research Software Engineer,"Burlington, VT",4d,$98K - $209K (Employer est.),98,209,NaN,Employer
631,"VIZIO, Inc.",3.4,PHP Engineer,"Tampa, FL",30d+,$74K - $92K (Employer est.),74,92,NaN,Employer
...,...,...,...,...,...,...,...,...,...,...
242,IdolSoft,3.3,SOFTWARE DEVELOPER,"Irving, TX",30d+,$104K - $127K (Glassdoor est.),104,127,NaN,Glassdoor
176,Artera Technologies,4.4,Cloud Software Engineers,United States,6d,$147K - $189K (Employer est.),147,189,NaN,Employer
457,Cisco Systems,4.5,Software Engineer,"Jersey City, NJ",4d,$87K - $124K (Glassdoor est.),87,124,NaN,Glassdoor
809,McKesson,NaN,Software Engineer - Entertainment Controls,"Middleton, WI",30d+,NaN,NaN,NaN,NaN,NaN


Drop NaN Salaries

In [293]:
df = df[df['MIN'].notna()]
df['MAX'].fillna(df['MIN'],inplace=True)
df.sample(100)

,Company,Company Score,Job Title,Location,Date,Salary,MIN,MAX,Hourly,Source
730,Federal Reserve Bank of Boston,4.1,Experienced Front-End Engineer,"Fort Collins, CO",14d,$81K - $109K (Glassdoor est.),81.0,109.0,False,Glassdoor
286,Whisker,4.3,Systems Software Engineer,"New York, NY",30d+,$110K - $190K (Employer est.),110.0,190.0,False,Employer
743,Teradyne,3.6,Software Engineer (Onsite),"Columbia, MD",25d,$248K - $385K (Employer est.),248.0,385.0,False,Employer
545,Root//Source,3.5,Software Engineer - Rider Support Tools,"Foster City, CA",30d+,$99.00 Per Hour (Employer est.),99.0,99.0,True,Employer
630,Harley-Davidson,3.8,Software Engineer 2,United States,30d+,$98K - $147K (Glassdoor est.),98.0,147.0,False,Glassdoor
...,...,...,...,...,...,...,...,...,...,...
171,"Benistar Administrative Services, Inc.",3.9,AWS Cloud Engineer,"Avon, CT",2d,$86K - $131K (Glassdoor est.),86.0,131.0,False,Glassdoor
272,"Messenger, LLC",4.4,Software Development ITAM Engineer,"Lansing, MI",5d,$116K - $166K (Glassdoor est.),116.0,166.0,False,Glassdoor
517,Workday,5.0,"Senior Software Engineer, Service Mesh","Boston, MA",5d,$129K - $202K (Employer est.),129.0,202.0,False,Employer
283,Raytheon,4.0,Software Engineer,"Cambridge, MA",2d,$79K - $168K (Employer est.),79.0,168.0,False,Employer


Mark Per hour salaries

In [294]:
df['Hourly']=df['Hourly'].fillna(0)
df['Hourly']=df['Hourly'].replace(["Per Hour"],1)
df.sample(100)

,Company,Company Score,Job Title,Location,Date,Salary,MIN,MAX,Hourly,Source
183,University of Florida,3.2,Software Engineer I/II,Florida,16d,$90K - $116K (Glassdoor est.),90.0,116.0,False,Glassdoor
211,Cedar Inc,3.3,Software Engineer II (Communications Platform),"New York, NY",20d,$108K - $140K (Employer est.),108.0,140.0,False,Employer
121,Consumers Energy,4.8,Assoc Software Engineer,"Jackson, MI",6d,$90K - $124K (Glassdoor est.),90.0,124.0,False,Glassdoor
366,Innovative Defense Technologies (IDT),3.0,Software Engineer (One Year Only),"Wilton Woods, VA",30d+,$92K - $131K (Glassdoor est.),92.0,131.0,False,Glassdoor
144,WALGREENS,4.4,Software Engineer (RxR),"Deerfield, IL",1d,$76K - $121K (Glassdoor est.),76.0,121.0,False,Glassdoor
...,...,...,...,...,...,...,...,...,...,...
37,Hasbro,4.4,"Software Engineer, Unreal Engine","Renton, WA",18d,$87K - $116K (Glassdoor est.),87.0,116.0,False,Glassdoor
722,Hewlett Packard,3.8,Product Engineer I,"Longmont, CO",24h,$97K - $133K (Glassdoor est.),97.0,133.0,False,Glassdoor
248,CliftonLarsonAllen,4.8,DevOps Engineer,"Minneapolis, MN",11d,$94K - $156K (Employer est.),94.0,156.0,False,Employer
710,bellelynn,3.6,IT Software Engineer,"Boise, ID",30d+,$77K - $104K (Glassdoor est.),77.0,104.0,False,Glassdoor


Check for NaN values in "Location" 

In [299]:
df_tmp = df.loc[df['Location'].isnull()]
df_tmp.sample(11)

,Company,Company Score,Job Title,Location,Date,Salary,MIN,MAX,Hourly,Source
394,Proteus Co.,3.5,Engineer II - Software,NaN,2d,$56K - $89K (Glassdoor est.),56.0,89.0,False,Glassdoor
44,McKesson,3.9,Software Engineer,NaN,2d,$88K - $124K (Glassdoor est.),88.0,124.0,False,Glassdoor
97,Careerus Soultions,4.0,DevOps Engineer,NaN,2d,$90K - $125K (Glassdoor est.),90.0,125.0,False,Glassdoor
298,Numerica Corporation,4.7,Mobile Engineer,NaN,2d,$71K - $104K (Glassdoor est.),71.0,104.0,False,Glassdoor
304,Northwest Talent Solutions LLC,3.6,Software Engineer 2,NaN,2d,$93K - $131K (Glassdoor est.),93.0,131.0,False,Glassdoor
103,Diverse Career Solutions,3.7,Software Engineer,NaN,2d,$48K - $81K (Glassdoor est.),48.0,81.0,False,Glassdoor
360,IXL Learning,3.8,"Sr Software Engineer, Snowflake focus",NaN,2d,$54K - $83K (Employer est.),54.0,83.0,False,Employer
528,Uber,3.9,Entry Level Support Equipment Software Engineer,NaN,2d,$105.00 - $115.00 Per Hour (Employer est.),105.0,105.0,True,Employer
264,Flysoft,5.0,Software Engineer,NaN,2d,$78K - $112K (Glassdoor est.),78.0,112.0,False,Glassdoor
19,Megaport,3.4,Frontend Software Engineer,NaN,2d,$84K - $133K (Glassdoor est.),84.0,133.0,False,Glassdoor


In [302]:
df_tmp = df.loc[df['Company'] == 'Uber']
df_tmp.sample(2)              

,Company,Company Score,Job Title,Location,Date,Salary,MIN,MAX,Hourly,Source
278,Uber,3.8,Senior Software Engineer,"Santa Barbara, CA",5d,$97K - $197K (Employer est.),97.0,197.0,False,Employer
528,Uber,3.9,Entry Level Support Equipment Software Engineer,NaN,2d,$105.00 - $115.00 Per Hour (Employer est.),105.0,105.0,True,Employer


In [348]:
df_tmp = df.loc[df['Company'] == 'Uber']
# df_tmp.sample(2)
df_tmp.dropna()
df_tmp.sample(2)
# df.loc[df['Company'] == 'Uber'].fillna(value=df_tmp['Location'])

# df[df['Company'] == 'Uber'].sample(2)         

,Company,Company Score,Job Title,Location,Date,Salary,MIN,MAX,Hourly,Source
528,Uber,3.9,Entry Level Support Equipment Software Engineer,NaN,2d,$105.00 - $115.00 Per Hour (Employer est.),105.0,105.0,1,Employer
278,Uber,3.8,Senior Software Engineer,"Santa Barbara, CA",5d,$97K - $197K (Employer est.),97.0,197.0,0,Employer


Check the Data Set after cleaning

In [304]:
#General structure of DS
# df.info()
# df=df['Company'].to_string
#check if there are any null or missing values
# print(df.isna().sum())
# %% of DS
#print(df.isna().sum()/len(df)*100)
df['Company'] = df['Company'].astype(str)
df['MIN'] = df['MIN'].astype(float)
df['MAX'] = df['MAX'].astype(float)
df['Hourly'] = df['Hourly'].astype(int)
df['Source'] = df['Source'].astype(str)



# df.info()
print(df.dtypes)

print(df.isna().sum())

Company           object
Company Score    float64
Job Title         object
Location          object
Date              object
Salary            object
MIN              float64
MAX              float64
Hourly             int64
Source            object
dtype: object
Company           0
Company Score     0
Job Title         0
Location         11
Date              0
Salary            0
MIN               0
MAX               0
Hourly            0
Source            0
dtype: int64
